In [4]:
# Load Pytesseract and other libraries#
from PIL import Image, ImageFilter, ImageEnhance
import pytesseract
import pandas as pd
import numpy as np
pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\jonat\\Tesseract-OCR\\tesseract.exe'

# Open image and check it works, initialise variables #
filename = "PID sample 1.jpeg"
PID = Image.open(filename)
PID.show()
img = PID.convert("RGB")
dim = PID.size
height = dim[0]
width = dim[1]
print(dim)

factor = 3

# Sharpness filter 
img2 = img.filter(ImageFilter.SHARPEN)

# Create a contrast enhancer object
contrast_enhancer = ImageEnhance.Contrast(img2)
# Adjust contrast
sharp_img = contrast_enhancer.enhance(factor)
sharp_img.show()

# Stream detector
# Initialise dictionaries
RGBvals = {}
streams = {}
colours = {}
counters = {}

# Load target variables from an excel sheet with colours for streams
target_streams = pd.read_excel('colours.xlsx')
rows = target_streams.shape[0]
# From excel sheet with list of coloured streams, load target colours and index match to colours
for i in range(rows):
    streams[i] = {}
    RGB = []
    for j in range(1,4):
        RGB.append(target_streams.loc[i][j])
    colours[i] = tuple(RGB)
    counters[i] = 0
colours[2] = (255,0,0)
colours[3] = (0,0,255)
print(colours)

# Find all matching points with the target colour, and input them into a nested dictionary corresponding to each stream
for i in range(height):
    for j in range(width):
        pixel = sharp_img.getpixel((i, j))
        RGBvals[(i, j)] = pixel
        for k in range(len(colours)):
            if RGBvals[(i,j)] == colours[k]:
                b = counters[k]
                streams[k][b] = tuple((i,j))
                counters[k] += 1

print(streams[2])

(1129, 737)
{0: (0, 255, 0), 1: (255, 153, 0), 2: (255, 0, 0), 3: (0, 0, 255)}
{0: (61, 536), 1: (61, 544), 2: (61, 545), 3: (61, 546), 4: (61, 547), 5: (62, 535), 6: (62, 536), 7: (62, 537), 8: (62, 538), 9: (62, 539), 10: (62, 540), 11: (62, 541), 12: (62, 542), 13: (62, 543), 14: (62, 544), 15: (62, 545), 16: (62, 546), 17: (62, 547), 18: (63, 535), 19: (63, 536), 20: (63, 537), 21: (63, 541), 22: (63, 542), 23: (63, 543), 24: (63, 544), 25: (63, 545), 26: (63, 546), 27: (63, 547), 28: (64, 535), 29: (64, 536), 30: (64, 537), 31: (64, 538), 32: (64, 539), 33: (64, 540), 34: (64, 541), 35: (64, 542), 36: (64, 543), 37: (64, 544), 38: (64, 545), 39: (64, 546), 40: (64, 547), 41: (65, 536), 42: (65, 537), 43: (65, 538), 44: (65, 539), 45: (65, 540), 46: (65, 541), 47: (65, 542), 48: (65, 543), 49: (65, 544), 50: (66, 536), 51: (66, 537), 52: (66, 540), 53: (66, 541), 54: (66, 542), 55: (66, 543), 56: (66, 544), 57: (66, 546), 58: (66, 547), 59: (67, 533), 60: (67, 536), 61: (67, 537), 

In [18]:
# Load Grid Size
filename2 = "PID with coloured instruments.tif"
file2 = Image.open(filename2)
PIDkey = file2.convert("RGB")
dim2 = file2.size
height2 = dim2[0]
width2 = dim2[1]
print(dim2)

gridcol = int(input("How many columns are there in your P&ID grid?"))
gridrow = int(input("How many rows are there in your P&ID grid?"))
boxheight = height2/gridrow
boxwidth = width2/gridcol

# Function to Convert Letters in Grid Reference to a box number
def letter_to_alphabetical_order(letter):
    if len(letter) != 1 or not letter.isalpha():
        return None  # Invalid input
        
    letter = letter.upper()  # Convert to uppercase
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    return alphabet.index(letter) + 1

# Load valve and sensor positions
valvesin = pd.read_excel('Valve list.xlsx')
valves = valvesin.to_numpy()
sensorsin = pd.read_excel('Indicator List.xlsx')
sensors = sensorsin.to_numpy()
print(len(sensors))

# Convert the box positions to a pixel reference
valvecoords = []
sensorcoords = []

for i in range(len(valves)):
    valvecoords.append([0,0,0,0])
    valvecoords[i][0] = valves[i,1]
    valvecoords[i][1] = int(boxheight * (letter_to_alphabetical_order(valves[i,3][0]) + 0.5))
    valvecoords[i][2] = int(boxwidth * (int(valves[i,3][1:]) + 0.5))
    
for j in range(len(sensors)):
    sensorcoords.append([0,0,0,0])
    sensorcoords[j][0] = sensors[j,0]
    sensorcoords[j][1] = int(boxheight * (letter_to_alphabetical_order(sensors[j,3][0]) + 0.5))
    sensorcoords[j][2] = int(boxwidth * (int(sensors[j,3][1:]) + 0.5))

# Find all the valves in the image
# Load target variables from an excel sheet with colours for streams - replace with colour chart of transmitters and valves
key = pd.read_excel('Indicator Colour Chart.xlsx')
rowno = key.shape[0]

RGBvals = {}
instruments = {}
colours2 = {}
counters2 = {}

for i in range(rowno):
    instruments[key.loc[i][4]] = {}
    RGB = []
    for j in range(1,4):
        RGB.append(key.loc[i][j])
    colours2[i] = tuple(RGB)
    counters2[i] = 0
print(instruments,colours,counters2)

# Find all matching points with the target colour, and input them into a nested dictionary corresponding to each stream
for i in range(1,3000):
    for j in range(1,2000):
        pixel = PIDkey.getpixel((i, j))
        RGBvals[(i, j)] = pixel
        for k in range(len(colours)):
            if RGBvals[(i,j)] == colours2[k]:
                b = counters2[k]
                instruments[k][b] = tuple((i,j))
                counters2[k] += 1

print(counters2)

(9362, 6623)


How many columns are there in your P&ID grid? 9
How many rows are there in your P&ID grid? 94


97
{'Manual Valve': {}, 'SOV': {}, 'FCV': {}, 'PCV': {}, 'RV': {}, 'XV': {}, 'FT': {}, 'TT': {}, 'LT': {}, 'PT': {}, 'AT': {}} {0: (255, 0, 0), 1: (0, 0, 255), 2: (255, 204, 1), 3: (0, 255, 0), 4: (255, 0, 255), 5: (0, 255, 255), 6: (128, 0, 128), 7: (0, 128, 128), 8: (192, 0, 64), 9: (128, 128, 0), 10: (64, 0, 192)} {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}


KeyError: 7

In [ ]:
# Contrast filter
factor = 3

# Sharpness filter 
img2 = img.filter(ImageFilter.SHARPEN)

# Create a contrast enhancer object
contrast_enhancer = ImageEnhance.Contrast(img2)
# Adjust contrast
sharp_img = contrast_enhancer.enhance(factor)
sharp_img.show()

file = sharp_img.save("Test2.jpg")
text_from_image = pytesseract.image_to_data(PID)
print(text_from_image)

In [ ]:
# TEST COLOURS!
colours[2] = (255,255,0)
colours[3] = (155,0,0)

text_from_image = pytesseract.image_to_data(Image.open("Test2.jpg"))
print(text_from_image)

# Contrast filter
factor = 1
img = PID.convert("RGB")
# Create a contrast enhancer object
contrast_enhancer = ImageEnhance.Contrast(img)
# Adjust contrast
sharp_img = contrast_enhancer.enhance(factor)
sharp_img.show()

# Contrast filter
factor = 3
img = PID.convert("RGB")

# Sharpness filter 
img2 = img.filter(ImageFilter.SHARPEN)

# Create a contrast enhancer object
contrast_enhancer = ImageEnhance.Contrast(img2)
# Adjust contrast
sharp_img = contrast_enhancer.enhance(factor)
sharp_img.show()

# Find all matching points with the target colour, and input them into a nested dictionary corresponding to each stream
for i in range(height):
    for j in range(width):
        pixel = sharp_img.getpixel((i, j))
        RGBvals[(i, j)] = pixel
        for k in range(len(colours)):
            if RGBvals[(i,j)] == colours[k]:
                b = counters[k]
                streams[k][b] = tuple((i,j))
                counters[k] += 1

